#Import data & create sample with subset of data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/gdrive/MyDrive/Colab Notebooks/cats_vs_dogs_small'

In [3]:
import os
import shutil
import random
import pathlib

# Base directory
new_base_dir = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/cats_vs_dogs_small/cats_vs_dogs_small")

# Function to create a sample of images
def create_sample(source_dir, target_dir, sample_size):
    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    for category in ['cats', 'dogs']:
        src_path = os.path.join(source_dir, category)
        target_path = os.path.join(target_dir, category)
        os.makedirs(target_path, exist_ok=True)

        # Get a list of all image files
        files = os.listdir(src_path)
        random.shuffle(files)

        # Select the desired number of files
        selected_files = files[:sample_size // 2]  # Divide evenly between cats and dogs

        for file in selected_files:
            shutil.copy(os.path.join(src_path, file), os.path.join(target_path, file))

# Create training, validation, and test samples
create_sample(new_base_dir / 'train', '/content/sample/train', 1000)
create_sample(new_base_dir / 'validation', '/content/sample/validation', 500)
create_sample(new_base_dir / 'test', '/content/sample/test', 500)

# Load data using TensorFlow
import tensorflow as tf

train_data = tf.keras.utils.image_dataset_from_directory(
    '/content/sample/train', batch_size=32, image_size=(180, 180), label_mode='int'
)

val_data = tf.keras.utils.image_dataset_from_directory(
    '/content/sample/validation', batch_size=32, image_size=(180, 180), label_mode='int'
)

test_data = tf.keras.utils.image_dataset_from_directory(
    '/content/sample/test', batch_size=32, image_size=(180, 180), label_mode='int'
)

# Check the data structure
print(f"Train dataset: {len(train_data)} batches")
print(f"Validation dataset: {len(val_data)} batches")
print(f"Test dataset: {len(test_data)} batches")


Found 1000 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Train dataset: 32 batches
Validation dataset: 16 batches
Test dataset: 16 batches


#Create base model (model1) from scratch

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model1 = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 178, 178, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 89, 89, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 87, 87, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 43, 43, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 41, 41, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 18, 18, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 9, 9, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 7, 7, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │          12,545 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 991,041 (3.78 MB)

 Trainable params: 991,041 (3.78 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model1.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

#Model1 (base model) performance before any steps to prevent overfitting

In [7]:
import matplotlib.pyplot as plt

# Compile model
model1.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',   # Metric to monitor
    patience=3,           # Number of epochs to wait before stopping
    restore_best_weights=True
)

# Train model and capture history
history = model1.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stopping]
)

Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.5006 - loss: 0.7057 - val_accuracy: 0.6060 - val_loss: 0.6929
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.5482 - loss: 0.6981 - val_accuracy: 0.5020 - val_loss: 0.6903
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.5415 - loss: 0.6959 - val_accuracy: 0.5000 - val_loss: 0.6895
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.5841 - loss: 0.6849 - val_accuracy: 0.6180 - val_loss: 0.6682
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.6023 - loss: 0.6861 - val_accuracy: 0.5040 - val_loss: 0.7823
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.6218 - loss: 0.6441 - val_accuracy: 0.6020 - val_loss: 0.6867
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.6636 - loss: 0.6268 - val_accuracy: 0.5560 - val_loss: 0.8031


In [8]:
# Evaluate on test data
test_loss, test_acc = model1.evaluate(test_data)
print(f"Test accuracy: {test_acc:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 13s 827ms/step - accuracy: 0.5886 - loss: 0.6746
Test accuracy: 0.6260


#Data augmentation and dropout added to the model

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_data2 = train_datagen.flow_from_directory(
    '/content/sample/train',
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary'
)

val_data2 = val_test_datagen.flow_from_directory(
    '/content/sample/validation',
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary'
)

test_data2 = val_test_datagen.flow_from_directory(
    '/content/sample/test',
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [3]:
from tensorflow.keras import layers, models

model2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),  # Dropout applied here

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout applied here

    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Compile model
model2.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',   # Metric to monitor
    patience=3,           # Number of epochs to wait before stopping
    restore_best_weights=True
)

# Train model and capture history
history2 = model2.fit(
    train_data2,
    validation_data=val_data2,
    epochs=30,
    callbacks=[early_stopping]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 93s 3s/step - accuracy: 0.4807 - loss: 0.8862 - val_accuracy: 0.5160 - val_loss: 0.6922
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 93s 3s/step - accuracy: 0.5294 - loss: 0.6947 - val_accuracy: 0.5480 - val_loss: 0.6892
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step - accuracy: 0.5329 - loss: 0.6956 - val_accuracy: 0.5800 - val_loss: 0.6868
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.5745 - loss: 0.6882 - val_accuracy: 0.5660 - val_loss: 0.6697
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.5436 - loss: 0.6901 - val_accuracy: 0.6020 - val_loss: 0.6557
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.5770 - loss: 0.6751 - val_accuracy: 0.6320 - val_loss: 0.6448
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.6343 - loss: 0.6576 - val_accuracy: 0.6540 - val_loss: 0.6274
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.6135 - loss: 0.6432 - val_accuracy: 0.6660 - val_loss:

In [5]:
# Evaluate on test data
test_loss, test_acc = model2.evaluate(test_data2)
print(f"Test accuracy: {test_acc:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 726ms/step - accuracy: 0.6511 - loss: 0.6264
Test accuracy: 0.6780


#Increase training sample size to 1500 - validation and test data the same


In [10]:
import random
import shutil
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import pathlib

# Base directory - Redefine new_base_dir here
new_base_dir = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/cats_vs_dogs_small/cats_vs_dogs_small")


def create_sample(source_dir, target_dir, sample_size):
    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    for category in ['cats', 'dogs']:
        src_path = os.path.join(source_dir, category)
        target_path = os.path.join(target_dir, category)
        os.makedirs(target_path, exist_ok=True)

        # Get a list of all image files
        files = os.listdir(src_path)
        random.shuffle(files)

        # Select the desired number of files
        selected_files = files[:sample_size // 2]  # Divide evenly between cats and dogsimport os

        for file in selected_files:
            shutil.copy(os.path.join(src_path, file), os.path.join(target_path, file))

# Create training sample
create_sample(new_base_dir / 'train', '/content/sample2/train', 1500)


# Load data using TensorFlow
import tensorflow as tf

train_data3 = tf.keras.utils.image_dataset_from_directory(
    '/content/sample2/train', batch_size=32, image_size=(180, 180), label_mode='binary'
)

val_data2 = tf.keras.utils.image_dataset_from_directory(
    '/content/sample/validation', batch_size=32, image_size=(180, 180), label_mode='binary'
)

test_data2 = tf.keras.utils.image_dataset_from_directory(
    '/content/sample/test', batch_size=32, image_size=(180, 180), label_mode='binary'
)

# Check the data structure
print(f"Train dataset: {len(train_data3)} batches")
print(f"Validation dataset: {len(val_data2)} batches")
print(f"Test dataset: {len(test_data2)} batches")


# Data augmentation to reduce overfitting
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

# Build the CNN model
model3 = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Rescaling(1./255), # Normalize pixel values
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout to prevent overfitting
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model3.compile(
    optimizer="rmsprop",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6)
model_checkpoint = ModelCheckpoint(filepath='best_model3.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history3 = model3.fit(
    train_data3,
    validation_data=val_data2,
    epochs=30,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Evaluate on test data
test_loss, test_acc = model3.evaluate(test_data2)
print(f"Test accuracy: {test_acc:.4f}")

Found 1991 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Train dataset: 63 batches
Validation dataset: 16 batches
Test dataset: 16 batches
Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4786 - loss: 1.2947

63/63 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.4788 - loss: 1.2881 - val_accuracy: 0.5000 - val_loss: 0.6978 - learning_rate: 0.0010
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5209 - loss: 0.6951

63/63 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.5210 - loss: 0.6951 - val_accuracy: 0.5040 - val_loss: 0.6925 - learning_rate: 0.0010
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 191s 3s/step - accuracy: 0.5356 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 0.8202 - learning_rate: 0.0010
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5075 - loss: 0.7084

63/63 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - accuracy: 0.5076 - loss: 0.7082 - val_accuracy: 0.5380 - val_loss: 0.6852 - learning_rate: 0.0010
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5589 - loss: 0.6835

63/63 ━━━━━━━━━━━━━━━━━━━━ 203s 3s/step - accuracy: 0.5589 - loss: 0.6835 - val_accuracy: 0.5980 - val_loss: 0.6610 - learning_rate: 0.0010
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 174s 3s/step - accuracy: 0.5978 - loss: 0.6668 - val_accuracy: 0.5960 - val_loss: 0.6554 - learning_rate: 0.0010
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5813 - loss: 0.6731

63/63 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.5815 - loss: 0.6730 - val_accuracy: 0.6600 - val_loss: 0.6089 - learning_rate: 0.0010
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accuracy: 0.6380 - loss: 0.6397 - val_accuracy: 0.6240 - val_loss: 0.6343 - learning_rate: 0.0010
Epoch 9/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 172s 3s/step - accuracy: 0.6390 - loss: 0.6298 - val_accuracy: 0.6120 - val_loss: 0.6660 - learning_rate: 0.0010
Epoch 10/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.6461 - loss: 0.6304 - val_accuracy: 0.5380 - val_loss: 1.1524 - learning_rate: 0.0010
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 676ms/step - accuracy: 0.6823 - loss: 0.6024
Test accuracy: 0.6620


#Increase training sample size to 2000 - validation and test data the same


In [11]:
import random
import shutil
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def create_sample(source_dir, target_dir, sample_size):
    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    for category in ['cats', 'dogs']:
        src_path = os.path.join(source_dir, category)
        target_path = os.path.join(target_dir, category)
        os.makedirs(target_path, exist_ok=True)

        # Get a list of all image files
        files = os.listdir(src_path)
        random.shuffle(files)

        # Select the desired number of files
        selected_files = files[:sample_size // 2]  # Divide evenly between cats and dogsimport os

        for file in selected_files:
            shutil.copy(os.path.join(src_path, file), os.path.join(target_path, file))

# Create training, validation, and test samples
create_sample(new_base_dir / 'train', '/content/sample3/train', 2000)


# Load data using TensorFlow
import tensorflow as tf

train_data4 = tf.keras.utils.image_dataset_from_directory(
    '/content/sample3/train', batch_size=32, image_size=(180, 180), label_mode='binary'
)

val_data2 = tf.keras.utils.image_dataset_from_directory(
    '/content/sample/validation', batch_size=32, image_size=(180, 180), label_mode='binary'
)

test_data2 = tf.keras.utils.image_dataset_from_directory(
    '/content/sample/test', batch_size=32, image_size=(180, 180), label_mode='binary'
)

# Check the data structure
print(f"Train dataset: {len(train_data4)} batches")
print(f"Validation dataset: {len(val_data2)} batches")
print(f"Test dataset: {len(test_data2)} batches")


# Data augmentation to reduce overfitting
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

# Build the CNN model
model4 = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Rescaling(1./255), # Normalize pixel values
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout to prevent overfitting
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model4.compile(
    optimizer="rmsprop",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6)
model_checkpoint = ModelCheckpoint(filepath='best_model4.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history4 = model4.fit(
    train_data4,
    validation_data=val_data2,
    epochs=30,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Evaluate on test data
test_loss, test_acc = model4.evaluate(test_data2)
print(f"Test accuracy: {test_acc:.4f}")

Found 2000 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Train dataset: 63 batches
Validation dataset: 16 batches
Test dataset: 16 batches
Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4812 - loss: 0.9057

63/63 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.4814 - loss: 0.9033 - val_accuracy: 0.5000 - val_loss: 0.6942 - learning_rate: 0.0010
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5183 - loss: 0.6954

63/63 ━━━━━━━━━━━━━━━━━━━━ 179s 3s/step - accuracy: 0.5183 - loss: 0.6954 - val_accuracy: 0.6040 - val_loss: 0.6867 - learning_rate: 0.0010
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5669 - loss: 0.6894

63/63 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.5669 - loss: 0.6894 - val_accuracy: 0.6100 - val_loss: 0.6428 - learning_rate: 0.0010
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.5943 - loss: 0.6686 - val_accuracy: 0.6020 - val_loss: 0.6496 - learning_rate: 0.0010
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6295 - loss: 0.6479

63/63 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.6294 - loss: 0.6479 - val_accuracy: 0.6200 - val_loss: 0.6188 - learning_rate: 0.0010
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 193s 3s/step - accuracy: 0.6395 - loss: 0.6353 - val_accuracy: 0.6200 - val_loss: 0.6403 - learning_rate: 0.0010
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6683 - loss: 0.6192

63/63 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.6682 - loss: 0.6194 - val_accuracy: 0.6520 - val_loss: 0.6068 - learning_rate: 0.0010
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6628 - loss: 0.6079

63/63 ━━━━━━━━━━━━━━━━━━━━ 203s 3s/step - accuracy: 0.6628 - loss: 0.6080 - val_accuracy: 0.6540 - val_loss: 0.6031 - learning_rate: 0.0010
Epoch 9/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6812 - loss: 0.5908

63/63 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.6812 - loss: 0.5910 - val_accuracy: 0.6740 - val_loss: 0.5837 - learning_rate: 0.0010
Epoch 10/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6838 - loss: 0.5895

63/63 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accuracy: 0.6836 - loss: 0.5898 - val_accuracy: 0.6940 - val_loss: 0.5747 - learning_rate: 0.0010
Epoch 11/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 213s 3s/step - accuracy: 0.7032 - loss: 0.5807 - val_accuracy: 0.6820 - val_loss: 0.5802 - learning_rate: 0.0010
Epoch 12/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.7074 - loss: 0.5821 - val_accuracy: 0.6780 - val_loss: 0.5886 - learning_rate: 0.0010
Epoch 13/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.6894 - loss: 0.5739 - val_accuracy: 0.6300 - val_loss: 0.6405 - learning_rate: 0.0010
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 668ms/step - accuracy: 0.6675 - loss: 0.5910
Test accuracy: 0.6880


#Add in pretrained network

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Load the VGG16 model without the top layer
base_model = VGG16(
    weights='imagenet',  # Load pretrained weights from ImageNet
    include_top=False,   # Remove fully connected layer
    input_shape=(180, 180, 3)  # Image size for VGG16
)

# Freeze the convolutional base to prevent retraining
base_model.trainable = False

# Create a new model on top of VGG16
model5 = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(180, activation='relu'),
    layers.Dropout(0.5),  # Dropout to reduce overfitting
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model5.compile(
    optimizer="rmsprop",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, min_lr=1e-6)
model_checkpoint = ModelCheckpoint(filepath='best_vgg16_model5.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history5 = model5.fit(
    train_data2,
    validation_data=val_data2,
    epochs=10,  # Train for 10 epochs first
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Evaluate on test data
test_loss, test_acc = model5.evaluate(test_data2)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.6010 - loss: 1.7329 

32/32 ━━━━━━━━━━━━━━━━━━━━ 611s 19s/step - accuracy: 0.6022 - loss: 1.7170 - val_accuracy: 0.8340 - val_loss: 5.9638 - learning_rate: 0.0010
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.7124 - loss: 0.6113 

32/32 ━━━━━━━━━━━━━━━━━━━━ 602s 19s/step - accuracy: 0.7129 - loss: 0.6104 - val_accuracy: 0.8400 - val_loss: 7.1310 - learning_rate: 0.0010
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.7601 - loss: 0.5158 

32/32 ━━━━━━━━━━━━━━━━━━━━ 607s 19s/step - accuracy: 0.7600 - loss: 0.5157 - val_accuracy: 0.8500 - val_loss: 8.8246 - learning_rate: 0.0010
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 605s 19s/step - accuracy: 0.7777 - loss: 0.5005 - val_accuracy: 0.8060 - val_loss: 18.9332 - learning_rate: 0.0010
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 603s 19s/step - accuracy: 0.8062 - loss: 0.4441 - val_accuracy: 0.8260 - val_loss: 15.9305 - learning_rate: 0.0010
16/16 ━━━━━━━━━━━━━━━━━━━━ 197s 12s/step - accuracy: 0.8673 - loss: 6.6291
Test accuracy: 0.8500


In [16]:
# Load the VGG16 model without the top layer
base_model = VGG16(
    weights='imagenet',  # Load pretrained weights from ImageNet
    include_top=False,   # Remove fully connected layer
    input_shape=(180, 180, 3)  # Image size for VGG16
)

# Freeze the convolutional base to prevent retraining
base_model.trainable = False

# Create a new model on top of VGG16
model6 = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(180, activation='relu'),
    layers.Dropout(0.5),  # Dropout to reduce overfitting
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model6.compile(
    optimizer="rmsprop",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, min_lr=1e-6)
model_checkpoint = ModelCheckpoint(filepath='best_vgg16_model6.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history6 = model6.fit(
    train_data3,
    validation_data=val_data2,
    epochs=10,  # Train for 10 epochs first
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Evaluate on test data
test_loss, test_acc = model6.evaluate(test_data2)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.8114 - loss: 15.1456 

63/63 ━━━━━━━━━━━━━━━━━━━━ 1046s 17s/step - accuracy: 0.8125 - loss: 14.9938 - val_accuracy: 0.9160 - val_loss: 0.8590 - learning_rate: 0.0010
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1038s 16s/step - accuracy: 0.9260 - loss: 0.5173 - val_accuracy: 0.9140 - val_loss: 0.6881 - learning_rate: 0.0010
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 989s 16s/step - accuracy: 0.9618 - loss: 0.2748 - val_accuracy: 0.9160 - val_loss: 0.6935 - learning_rate: 0.0010
16/16 ━━━━━━━━━━━━━━━━━━━━ 197s 12s/step - accuracy: 0.9250 - loss: 0.7312
Test accuracy: 0.9240


In [17]:
# Load the VGG16 model without the top layer
base_model = VGG16(
    weights='imagenet',  # Load pretrained weights from ImageNet
    include_top=False,   # Remove fully connected layer
    input_shape=(180, 180, 3)  # Image size for VGG16
)

# Freeze the convolutional base to prevent retraining
base_model.trainable = False

# Create a new model on top of VGG16
model7 = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(180, activation='relu'),
    layers.Dropout(0.5),  # Dropout to reduce overfitting
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model7.compile(
    optimizer="rmsprop",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, min_lr=1e-6)
model_checkpoint = ModelCheckpoint(filepath='best_vgg16_model7.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history7 = model7.fit(
    train_data4,
    validation_data=val_data2,
    epochs=10,  # Train for 10 epochs first
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Evaluate on test data
test_loss, test_acc = model7.evaluate(test_data2)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.8063 - loss: 20.7833 

63/63 ━━━━━━━━━━━━━━━━━━━━ 989s 16s/step - accuracy: 0.8075 - loss: 20.5870 - val_accuracy: 0.9420 - val_loss: 0.6198 - learning_rate: 0.0010
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1054s 16s/step - accuracy: 0.9556 - loss: 0.4802 - val_accuracy: 0.9380 - val_loss: 0.5718 - learning_rate: 0.0010
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1035s 16s/step - accuracy: 0.9687 - loss: 0.3511 - val_accuracy: 0.9260 - val_loss: 0.6661 - learning_rate: 0.0010
16/16 ━━━━━━━━━━━━━━━━━━━━ 196s 12s/step - accuracy: 0.9614 - loss: 0.5031
Test accuracy: 0.9560
